import cv2

In [1]:
import cv2

In [10]:
# Initialize video capture (0 for webcam)
cap = cv2.VideoCapture(0)

# Get the video frame width, height, and frames per second (fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 20  # Typically, webcam fps can be manually set, 20 is a common default value

# Define the codec and create a VideoWriter object for the background model
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
output_video_path = 'background_retrieval_output.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height), isColor=True)

# Create a background subtractor using GMM
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=50, varThreshold=16, detectShadows=True)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    # Apply the background subtractor to get the foreground mask
    fg_mask = bg_subtractor.apply(frame)

    # Optional: Improve the mask using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    # Retrieve the background model
    background_model = bg_subtractor.getBackgroundImage()

    # If the background model is ready, write it to the output video
    if background_model is not None:
        out.write(background_model)

    # Display the results (optional, can be commented out)
    cv2.imshow('Foreground Mask', fg_mask)
    cv2.imshow('Background Model', background_model)

    if cv2.waitKey(30) & 0xFF == 27:  # Press 'ESC' to exit
        break

# Release the video capture and writer, and close all windows
cap.release()
out.release()
cv2.destroyAllWindows()
